**CATEGORIZACIÓN DE FENÓMENOS CARDÍACOS**

In [ ]:
#Librerías
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import missingno
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff
import scipy as sp
import scipy.fftpack
from scipy.signal import freqz
import os
import pylab
import scipy.signal as signal
import numpy 

In [ ]:
df=pd.read_csv("/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv")
df.head()  #Tomamos las 5 primeras filas

In [ ]:
# Crear una matriz para ver los valores faltantes
missingno.matrix(df, fontsize = 16)
plt.show()

In [ ]:
#Mirar el tamaño que tienen estos datos
print(f'La base de datos contiene {df.shape[0]} Filas y {df.shape[1]} Columnas')

In [ ]:
#Comprobar datos duplicados
df.duplicated().sum()

In [ ]:
#Contar la edad de los pacientes
plt.figure(figsize=(20,10))
sns.countplot(x=df["age"])
plt.title("Promedio de edad de los pacientes",fontsize=20)
plt.xlabel("EDAD",fontsize=20)
plt.ylabel("PROMEDIO",fontsize=20)
plt.show()

In [ ]:
#Porcentaje de edad, hombres y mujeres
s=df["sex"].value_counts().reset_index()
px.pie(s,names="index",values="sex",title="Porcentaje de edad de hombres y mujeres:")

In [ ]:
#Nivel de dolor por cantidad de pacientes
c=df["cp"].value_counts().reset_index()
plt.figure(figsize=(20,10))
sns.barplot(x=c["index"],y=c["cp"])
plt.title("NIVEL DE DOLOR VS CANTIDAD DE PACIENTES",fontsize=20)
plt.xlabel("NIVEL",fontsize=20)
plt.ylabel("CANTIDAD",fontsize=20)
plt.show()

In [ ]:
#Presión sanguínea en pacientes
sns.displot(df["trtbps"])
plt.title("DISTRIBUCIÓN DE LA PRESIÓN SANGUÍNEA EN PACIENTES",fontsize=20)
plt.xlabel("PRESIÓN SANGUÍNEA",fontsize=10)
plt.ylabel("PACIENTES",fontsize=10)
plt.show()

In [ ]:
#Nivel de colestero en pacientes
sns.displot(df["chol"])
plt.title("DISTRIBUCIÓN DE NIVEL DE COLESTEROL EN PACIENTES",fontsize=20)
plt.xlabel("NIVEL DE COLESTEROL",fontsize=10)
plt.ylabel("PACIENTES",fontsize=10)
plt.show()

In [ ]:
#Ritmo cardíaco en pacientes
sns.displot(df["thalachh"])
plt.title("DISTRIBUCIÓN DEL RITMO CARDÍACO EN PACIENTES",fontsize=20)
plt.xlabel("RITMO CARDÍACO",fontsize=10)
plt.ylabel("PACIENTES",fontsize=10)
plt.show()

In [ ]:
#Influencia de la edad en infartos
plt.figure(figsize=(10,6))
sns.histplot(data = df, x = 'age', hue = 'output')
plt.title("¿LA EDAD INFLUYE EN LOS INFARTOS?")
plt.xlabel("EDAD",fontsize=10)
plt.ylabel("PACIENTES",fontsize=10)
plt.show()

In [ ]:
#Taquicardia con edad
v=pd.crosstab(df["age"],df["output"]).reset_index()

v.columns=["age","low_risk","high_risk"]

px.line(v,v["age"],v["high_risk"],title="RIESGO DE TAQUICARDIA RESPECTO A LA EDAD")

In [ ]:
#Bradicardia con edad
px.line(v,v["age"],v["low_risk"],title="RIESGO DE BRADICARDIA CON RESPECTO A LA EDAD")

In [ ]:
#Presión sanguínea respecto a la edad
plt.figure(figsize=(20,10))
sns.lineplot(y="trtbps",x="age",data=df)
plt.title("PRESION SANGUÍNEA RESPECTO A LA EDAD",fontsize=20)
plt.xlabel("EDAD",fontsize=20)
plt.ylabel("PRESIÓN SANGUÍNEA",fontsize=20)
plt.show()

In [ ]:
# Creación de Señal simulada
Fs = 44100 # Frecuencia de muestreo
t_w = 1    # Duración de la señal en segundos
t = np.linspace(0, t_w, Fs*t_w, endpoint = False)
f1 = 500  # Frecuencia en Hz
f2 = 4000
sig1 = 2*np.sin(2*np.pi*f1*t) 
sig2 = 0.5*np.sin(2*np.pi*f2*t)
sig = sig1 + sig2 

# Cálculo del espectro de la señal
yf = scipy.fftpack.fft(sig)                    # Transformada de Fourier de la señal
Espec = 2.0 / len(t) * np.abs( yf [ 0:int(len(t)/2) ] )   # Espectro de la señal
Freq = np.linspace(0.0,  Fs/2 , int(len(t)/2) )      #Vector de frecuencias del espectro

# Plot de señal y su composición espectral
plt.figure()
plt.plot(t,sig),plt.grid(),plt.xlabel("Time (seconds)"), plt.title("Señal Simulada")
plt.xlim([0,0.01])
plt.figure()

In [ ]:
#btype{‘lowpass’, ‘highpass’, ‘bandpass’, ‘bandstop’} 
from scipy.signal import butter, lfilter

# FILTRO PASABANDA
def butter_bandpass(lowcut, highcut, Fs, order = 5):
  nyq = Fs / 2.
  low = lowcut / nyq
  high = highcut / nyq
  b, a = butter(order, [low, high], btype='bandpass')
  return b, a

def butter_bandpass_filter(data, lowcut, highcut, Fs, order = 5):
  b, a = butter_bandpass(lowcut, highcut, Fs, order=order)
  y = lfilter(b, a, data)
  w, h = freqz(b, a, worN=2000)
  plt.figure()
  plt.plot((Fs * 0.5 / np.pi) * w, abs(h), label="order = %d" % order)
  plt.xlabel('Frequency (Hz)'),plt.ylabel('Gain'),plt.grid(),plt.legend(loc='best')
  plt.figure()
  angles = np.unwrap(np.angle(h))
  plt.plot((Fs * 0.5 / np.pi) * w, angles, 'g')
  plt.ylabel('Angle (radians)', color='g')
  plt.xlabel('Frequency (Hz)')
  plt.grid()
  plt.axis('tight')
  return y

# FILTRO PASABAJAS
def butter_lowpass(freq_cut, Fs, order=5):
  nyq = Fs / 2.
  freq_c = freq_cut / nyq
  b, a = butter(order, freq_c, btype='lowpass')
  return b, a
def butter_lowpass_filter(data, freq_cut, Fs, order=5):
  b, a = butter_lowpass(freq_cut, Fs, order=order)
  y = lfilter(b, a, data)
  w, h = freqz(b, a, worN=2000)
  plt.figure()
  plt.plot((Fs * 0.5 / np.pi) * w, abs(h), label="order = %d" % order)
  plt.xlabel('Frequency (Hz)'),plt.ylabel('Gain'),plt.grid(),plt.legend(loc='best')
  plt.figure()
  angles = np.unwrap(np.angle(h))
  plt.plot((Fs * 0.5 / np.pi) * w, angles, 'g')
  plt.ylabel('Angle (radians)', color='g')
  plt.xlabel('Frequency (Hz)')
  plt.grid()
  plt.axis('tight')
  return y

# FILTRO PASAALTAS
def butter_highpass(freq_cut, Fs, order=5):
  nyq = Fs / 2.
  freq_c = freq_cut / nyq
  b, a = butter(order, freq_c, btype='highpass')
  return b, a
def butter_highpass_filter(data, freq_cut, Fs, order=5):
  b, a = butter_highpass(freq_cut, Fs, order=order)
  y = lfilter(b, a, data)
  w, h = freqz(b, a, worN=2000)
  plt.figure()
  plt.plot((Fs * 0.5 / np.pi) * w, abs(h), label="order = %d" % order)
  plt.figure()
  angles = np.unwrap(np.angle(h))
  plt.plot((Fs * 0.5 / np.pi) * w, angles, 'g')
  plt.ylabel('Angle (radians)', color='g')
  plt.xlabel('Frequency (Hz)')
  plt.grid()
  plt.axis('tight')
  plt.xlabel('Frequency (Hz)'),plt.ylabel('Gain'),plt.grid(),plt.legend(loc='best')
  return y

  # Filtro Rechaza Banda
  

In [ ]:
lowcut = 550.0 #Hz
highcut = 3000.0 #Hz

# Aplicación del filtro a la señal simulada
sig_f = butter_bandpass_filter(sig, lowcut, highcut, Fs, order=6)
plt.figure()
plt.plot(t, sig_f), plt.xlabel('Time (seconds)'), plt.grid()

plt.xlim([0,0.02])
plt.show()

# Cálculo del espectro de la señal
yf = scipy.fftpack.fft(sig_f)                    # Transformada de Fourier de la señal
Espec = 2.0 / len(t) * np.abs( yf [ 0:int(len(t)/2) ] )   # Espectro de la señal
Freq = np.linspace(0.0,  Fs/2 , int(len(t)/2) )      #Vector de frecuencias del espectro

plt.show()

In [ ]:
from scipy import signal
#pass_zero{True, False, ‘bandpass’, ‘lowpass’, ‘highpass’, ‘bandstop’},
order = 100
f_cut = 2000
b = scipy.signal.firwin(order + 1, f_cut , window='hamming', pass_zero="lowpass", fs=Fs)

sig_f = lfilter(b, 1, sig)
plt.figure()
plt.plot(t, sig_f), plt.xlabel('Time (seconds)'), plt.grid()
plt.plot(t, sig1)
plt.xlim([0,0.007])
plt.show()

In [ ]:
print('SIMULACIÓN DE UN ECG - CARDIACO')

pqrst = signal.wavelets.daub(10)

samples_rest = 10
zero_array = numpy.zeros(samples_rest, dtype=int)
pqrst_full = numpy.concatenate([pqrst,zero_array])

pylab.plot(pqrst_full)
pylab.xlabel('Número de muestra')
pylab.ylabel('Amplitud (normalizada)')
pylab.title('Señal del latido del corazón')
pylab.show()

bpm = 60
bps = bpm / 60

capture_length = 10

num_heart_beats = int(capture_length * bps)

ecg_template = numpy.tile(pqrst_full , num_heart_beats)

pylab.plot(ecg_template)
pylab.xlabel('Número de muestra')
pylab.ylabel('Amplitud (normalizada)')
pylab.title('ECG')
pylab.show()

noise = numpy.random.normal(0, 0.01, len(ecg_template))
ecg_template_noisy = noise + ecg_template

# Plot the noisy heart ECG template
pylab.plot(ecg_template_noisy)
pylab.xlabel('Número de muestra')
pylab.ylabel('Amplitud (normalizada)')
pylab.title('ECG con ruido Gaussiano')
pylab.show()

sampling_rate = 50.0
num_samples = sampling_rate * capture_length
ecg_sampled = signal.resample(ecg_template_noisy, num_samples) 

adc_bit_resolution = 1024
ecg =  adc_bit_resolution * ecg_sampled

pylab.plot(ecg)
pylab.xlabel('Sample number')
pylab.ylabel('bit value')
pylab.title('%d bpm ECG signal with gaussian noise sampled at %d Hz' %(bpm, sampling_rate) )
pylab.show()
pylab.plot(ecg_template_noisy)
pylab.plot(ecg_template)
pylab.xlabel('Sample number')
pylab.ylabel('bit value')
pylab.title('%d bpm ECG signal with gaussian noise sampled at %d Hz' %(bpm, sampling_rate) )
pylab.show()

In [ ]:
pylab.plot(ecg_template_noisy)
pylab.plot(ecg_template)
pylab.xlabel('Número de muestra')
pylab.ylabel('Amplitud(normalizado)')
pylab.title('%d ECG con ruido Gaussiano junto al ECG normal %d'%(bpm, sampling_rate) )
pylab.show()